In [ ]:
from ultralytics import YOLO
from sklearn.cluster import KMeans
import numpy as np
import cv2



# Cargar el modelo YOLO entrenado
model = YOLO("./runs/detect/train2/weights/best.pt")

# Cargar el video de entrada
video_path = "./assets/videos/dal-lac1.mp4"
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configurar el video de salida
output_path = "video_resultado.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

team_colors = {'A': np.array([0,0,0]), 'B': np.array([0,0,0])}
player_team_map = {}

def classifyTeam(color):
    if np.array_equal(team_colors['A'], np.array([0,0,0])):
        team_colors['A'] = color
        return 'A'
    elif np.array_equal(team_colors['B'], np.array([0,0,0])):
        if np.linalg.norm(color - team_colors['A']) < 100:
            return 'A'
        team_colors['B'] = color
        return 'B'
    else:
        if np.linalg.norm(color - team_colors['A']) < np.linalg.norm(color - team_colors['B']):
            return 'A'
        else:
            return 'B'
        
# Función para obtener el ROI del área central de la bbox
def get_central_roi(frame, x1, y1, x2, y2):
    # Coordenadas de la bbox
    width = x2 - x1
    height = y2 - y1

    # Definir subárea como proporción de la bbox
    sub_x1 = int(x1 + width * 0.2)  # Dejar un 20% de margen a los lados
    sub_x2 = int(x2 - width * 0.2)  # Dejar un 20% de margen a los lados
    sub_y1 = int(y1 + height * 0.4)  # Subárea empieza al 40% de la altura
    sub_y2 = int(y1 + height * 0.7)  # Subárea llega al 70% de la altura

    # Recortar y devolver el ROI
    roi = frame[sub_y1:sub_y2, sub_x1:sub_x2]
    return roi

# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar detección en el frame
    results = model(frame)

    # Dibujar las detecciones en el frame
    for result in results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if model.names[cls] in ['player'] and conf > 0.35:

            label = f"{model.names[cls]} {conf:.2f}"
            player_position = (int((x1 + x2) / 2), y2)
            player_position_label = f"x:{int(player_position[0])} y:{int(player_position[1])}"
            team = ''

            if model.names[cls] == 'player':
                player_roi = get_central_roi(frame, int(x1), int(y1), int(x2), int(y2))
                roi_hsv = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)

                pixels = roi_hsv.reshape(-1, 3)

                k = 2
                kmeans = KMeans(n_clusters=k, random_state=0).fit(pixels)
                dominant_colors = kmeans.cluster_centers_
                labels, counts = np.unique(kmeans.labels_, return_counts=True)

                dominant_cluster = labels[np.argmax(counts)]
                dominant_color = dominant_colors[dominant_cluster]

                team = classifyTeam(dominant_color)
                print(f"Color dominante: {dominant_color} - Equipo: {team}")

            bgr_color = team_colors[team].astype(np.uint8)
            bgr_color = cv2.cvtColor(bgr_color[np.newaxis, np.newaxis, :], cv2.COLOR_HSV2BGR)[0, 0]
            print(f"Color BGR: {bgr_color}")
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), bgr_color.tolist(), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 200, 100), 2)
            cv2.ellipse(frame, (int(player_position[0]), int(player_position[1])), (20, 10), 0, 0, 360, (0, 0, 255), -1)
            cv2.putText(frame, player_position_label, (int(player_position[0]) - 50, int(player_position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Detecciones', frame)

    # Escribir el frame procesado en el video de salida
    out.write(frame)

    # Esperar 1ms para salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")

